#  Прогнозирование заказов такси

Компания-перевозчик собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Постройте модель для такого предсказания.

Значение метрики *RMSE* на тестовой выборке должно быть не больше 48.

Необходимо:

1. Загрузить данные и выполнить их ресемплирование по одному часу.
2. Проанализировать данные.
3. Обучить разные модели с различными гиперпараметрами. Сделать тестовую выборку размером 10% от исходных данных.
4. Проверить данные на тестовой выборке и сделать выводы.

## Подготовка

### Библиотеки

Загружаю необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
import catboost as cb
from sklearn.neighbors import KNeighborsRegressor

### Данные

Загружаю датасет.

In [2]:
taxi = pd.read_csv('/datasets/taxi.csv', index_col=[0], parse_dates=[0])

In [3]:
taxi.info(), taxi.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
num_orders    26496 non-null int64
dtypes: int64(1)
memory usage: 414.0 KB


(None,
                      num_orders
 datetime                       
 2018-03-01 00:00:00           9
 2018-03-01 00:10:00          14
 2018-03-01 00:20:00          28
 2018-03-01 00:30:00          20
 2018-03-01 00:40:00          32)

26496 временных интервала по 10 минут с количеством заказов такси за этот интервал.

In [4]:
taxi.index.min(), taxi.index.max()

(Timestamp('2018-03-01 00:00:00'), Timestamp('2018-08-31 23:50:00'))

In [5]:
taxi.sort_index(inplace=True)

In [6]:
taxi.index.min(), taxi.index.max()

(Timestamp('2018-03-01 00:00:00'), Timestamp('2018-08-31 23:50:00'))

Данные следуют в хронологическом порядке.

### Ресемпл по 1 часу

In [7]:
taxi_rsml = taxi.resample('1H').sum()

### Добавление новых признаков

In [8]:
df = pd.DataFrame(data=taxi_rsml, index=taxi_rsml.index)

Создаю функцию для добавления признаков.

In [9]:
def make_features(data, max_lag, rolling_mean_size):
    
    df['year'] = data.index.year
    df['month'] = data.index.month
    df['day'] = data.index.day
    df['dayofweek'] = data.index.dayofweek
    df['hour'] = data.index.hour
    
    for lag in range(1, max_lag + 1):
        df['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    df['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()

Имеем ресэмпл по 1 часу, так что lag=24 будет сдвигать данные в течение суток, а среднее скользящее "6" будет отслеживать тренд временного ряда (в смысле утро, день, вечер, ночь)

In [10]:
make_features(taxi_rsml, 24, 6)

### Сплит

По условию, выделяю тестовую выборку 10% от исходного датасета.

Чтобы не смотреть в будущее, использую shuffle=FALSE.

In [11]:
train, test = train_test_split(df, shuffle=False, test_size=0.1, random_state=1024)
train = train.dropna()

In [12]:
train.index.max(), test.index.min()

(Timestamp('2018-08-13 13:00:00'), Timestamp('2018-08-13 14:00:00'))

Данные следуют друг за другом

In [14]:
train.shape, test.shape

((3950, 31), (442, 31))

In [15]:
X_train = train.drop(['num_orders'], axis=1)
y_train = train['num_orders']

X_test = test.drop(['num_orders'], axis=1)
y_test = test['num_orders']

In [16]:
X_train.index.min(), X_train.index.max(), X_test.index.min(), X_test.index.max()

(Timestamp('2018-03-02 00:00:00'),
 Timestamp('2018-08-13 13:00:00'),
 Timestamp('2018-08-13 14:00:00'),
 Timestamp('2018-08-31 23:00:00'))

Тренировочная выборка с 02 марта по 13 августа 2018 г. (5 мес. 11 сут.), продолжает ее тестовая до 31 августа 2018 г. (18 сут.).

Индесы не перемешаны, в будущее не заглядываем.

### Масштабирование

In [17]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Данные подготовлены.

## Анализ

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: H
Data columns (total 31 columns):
num_orders      4416 non-null int64
year            4416 non-null int64
month           4416 non-null int64
day             4416 non-null int64
dayofweek       4416 non-null int64
hour            4416 non-null int64
lag_1           4415 non-null float64
lag_2           4414 non-null float64
lag_3           4413 non-null float64
lag_4           4412 non-null float64
lag_5           4411 non-null float64
lag_6           4410 non-null float64
lag_7           4409 non-null float64
lag_8           4408 non-null float64
lag_9           4407 non-null float64
lag_10          4406 non-null float64
lag_11          4405 non-null float64
lag_12          4404 non-null float64
lag_13          4403 non-null float64
lag_14          4402 non-null float64
lag_15          4401 non-null float64
lag_16          4400 non-null float64
lag_17          4399 non-

Количество строк сократилось в 6 раз - логично.

In [19]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3950, 30), (3950,), (442, 30), (442,))

Разбивка корректная.

## Обучение

### Dummy

In [20]:
dr = DummyRegressor(strategy="mean")
dr.fit(X_train, y_train)
dr_pred = dr.predict(X_train)
dr_rmse = np.sqrt(mean_squared_error(dr_pred, y_train))
print('RMSE на манекене для train-выборки: {:.2f}'.format(dr_rmse))

RMSE на манекене для train-выборки: 38.68


### LinearRegression

In [21]:
lr = LinearRegression(n_jobs=-1)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_train)
lr_rmse = np.sqrt(mean_squared_error(lr_pred, y_train))
print('RMSE на Linear Regression для train-выборки: {:.2f}'.format(lr_rmse))

RMSE на Linear Regression для train-выборки: 25.70


### CatBoost Regressor

Для ускорения.

Использовались следующие параметры:

grid = {
    
    'iterations': [100, 300, 500],
    
    'learning_rate': [0.003, 0.1, 0.3],
    
    'depth': [4, 6, 8]

}

Грид нашёл лучшие:

'params': {'depth': 4, 'iterations': 300, 'learning_rate': 0.3

In [31]:
train_dataset = cb.Pool(X_train, y_train)
cbr = cb.CatBoostRegressor(loss_function='RMSE', random_state = 1024)

grid = {
    'iterations': [300],
    'learning_rate': [0.3],
    'depth': [4]
}
cbr.grid_search(grid, train_dataset, cv=TimeSeriesSplit(n_splits=5));

0:	loss: 30.8528433	best: 30.8528433 (0)	total: 5.18s	remaining: 0us
Estimating final quality...


In [32]:
cbr_pred = cbr.predict(X_train)
cbr_rmse = np.sqrt(mean_squared_error(cbr_pred, y_train))
print('RMSE на CatBoostRegressor для train-выборки: {:.2f}'.format(cbr_rmse))

RMSE на CatBoostRegressor для train-выборки: 12.15


### KNeighborsRegressor

In [24]:
knr = KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
knr.fit(X_train, y_train)
knr_pred = knr.predict(X_train)
knr_rmse = np.sqrt(mean_squared_error(knr_pred, y_train))
print('RMSE на KNeighborsRegressor для train-выборки: {:.2f}'.format(knr_rmse))

RMSE на KNeighborsRegressor для train-выборки: 17.33


## Тестирование

### Dummy

In [25]:
dr_pred_test = dr.predict(X_test)
dr_rmse_test = np.sqrt(mean_squared_error(dr_pred_test, y_test))
print('RMSE на манекене для тестовой выборки: {:.2f}'.format(dr_rmse_test))

RMSE на манекене для тестовой выборки: 84.65


### LinearRegression

In [26]:
lr_pred_test = lr.predict(X_test)
lr_rmse_test = np.sqrt(mean_squared_error(lr_pred_test, y_test))
print('RMSE на Linear Regression для тестовой выборки: {:.2f}'.format(lr_rmse_test))

RMSE на Linear Regression для тестовой выборки: 45.81


### CatBoost Regressor

In [27]:
cbr_pred_test = cbr.predict(X_test)
cbr_rmse_test = np.sqrt(mean_squared_error(cbr_pred_test, y_test))
print('RMSE на CatBoostRegressor для тестовой выборки: {:.2f}'.format(cbr_rmse_test))

RMSE на CatBoostRegressor для тестовой выборки: 43.38


### KNeighborsRegressor

In [28]:
knr_pred_test = knr.predict(X_test)
knr_rmse_test = np.sqrt(mean_squared_error(knr_pred_test, y_test))
print('RMSE на KNeighborsRegressor для тестовой выборки: {:.2f}'.format(knr_rmse_test))

RMSE на KNeighborsRegressor для тестовой выборки: 49.13


## Вывод

In [29]:
d = {
    'DummyRegressor': [dr_rmse, dr_rmse_test],
    'LinearRegression' : [lr_rmse, lr_rmse_test],
    'CatBoostRegressor' : [cbr_rmse, cbr_rmse_test],
    'KNeighborsRegressor' : [knr_rmse, knr_rmse_test]}
resume = pd.DataFrame(data=d, index=['RMSE train', 'RMSE test'], dtype=np.int64)

In [30]:
resume

,DummyRegressor,LinearRegression,CatBoostRegressor,KNeighborsRegressor
RMSE train,38.6829,25.6992,12.1514,17.3314
RMSE test,84.6499,45.8108,43.3787,49.1263


$$Вывод$$

$Подготовка:$
на этапе подготовки произведено разделение исходного датасета на тренировочную и тестовую выборку в соотношении 9:1, что, в результате, показало значительное улучшение RMSE на тесте (видимо, в валидационную выборку попали очень важные данные).

$Обучение/Тестирование:$
Для оценки результатов, в качестве отправной точки, использовался манекен - DummyRegressor.

В качестве реальных моделей использовались LinearRegression, CatBoostRegressor & KNeighborsRegressor. Все реальные модели отработали лучше манекена, в лидерах - CatBoostRegressor (RMSE на тренировочной выборке - 12.15, на тестовой - 43.37). 

Стоить отметить LinearRegression, RMSE на тренировочной выборке оказался хуже, чем у CatBoost и KNeighborsRegressor, однако на тесте значение RMSE не сильно отстает от CatBoost и не превышает порог 48.

Лучше всего с прогнозированием количества заказов такси на следующий час справилась CatBoost Regressor $RMSE = 43.37$